<a href="https://colab.research.google.com/github/nrohrbach/PocDatenkatalogBafu/blob/main/PoC_Datenkatalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proof of Concept Datenkatalog BAFU für neue Webseite auf Livingdocs
Auf der BAFU Webseite sollen künftig alle Daten, Indikatoren, Statistiken und Geodatenmodelle über einen zentralen Katalog gefunden werden. Die Inhalte des Katalogs stammen aus der API von opendata.swiss und von anderen Quellen.

In [1]:
import requests
import pandas as pd


In [67]:
# Filtermöglichkeiten des Katalogs vorbereiten

Thema =   ['Abfall',
           'Altlasten',
           'Bildung, Forschung, Innovation',
           'Biodiversität',
           'Biotechnologie',
           'Boden',
           'Chemikalien',
           'Elektrosmog und Licht',
           'Ernährung, Wohnen, Mobilität',
           'Gesundheit',
           'Internationales',
           'Klima',
           'Landschaft',
           'Lärm',
           'Luft',
           'Naturgefahren',
           'Recht',
           'Störfallvorsorge',
           'Umweltverträglichkeitsprüfung',
           'Wald & Holz',
           'Wasser',
           'Wirtschaft und Konsum'
           ]

Datentyp = ['Indikator',
            'Statistik',
            'Geodaten',
            'Geodatenmodell',
            'Monitoring',
            'Daten']

Monitoring = ['LFI',
              'NABO',
              'NABEL',
              'NADUF',
              'NAWA',
              'NAQUA',
              'LABES',
              'BDM',
              'SonBase',]

In [3]:
# Mapping der BAFU-Themen auf das Keyword bei opendata.swiss
def map_options(option):
  mapping = {
      'Abfall': 'abfall',
      'Altlasten': 'altlasten',
      'Bildung, Forschung, Innovation': 'bildung',
      'Biodiversität': 'biodiversitat',
      'Biotechnologie' : 'biotechnologie',
      'Boden' : 'boden',
      'Chemikalien' : 'chemikalien',
      'Elektrosmog und Licht' : 'elektrosmog',
      'Ernährung, Wohnen, Mobilität'  : 'ernaehrung',
      'Gesundheit'  : 'gesundheit',
      'Internationales'  : 'internationales',
      'Klima'  : 'klima',
      'Landschaft'  : 'landschaft',
      'Lärm'  : 'laerm',
      'Luft'  : 'luft',
      'Naturgefahren'  : 'naturgefahren',
      'Recht'  : 'recht',
      'Störfallvorsorge'  : 'storfallvorsorge',
      'Umweltverträglichkeitsprüfung'  : 'umweltvertraeglichkeitpruefung',
      'Wald & Holz'  : 'wald',
      'Wasser'  : 'wasser',
      'Wirtschaft und Konsum'  : 'wirtschaft'
  }
  return mapping.get(option, None) # Return None if option is not found


In [43]:
url = "https://opendata.swiss/api/3/action/package_search"
params = {
    "q": "organization:bundesamt-fur-umwelt-bafu",
    "rows": 1000  # Request a large number of rows to get all entries
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    if data['success']:
        packages = data['result']['results']
        print(f"Found {len(packages)} entries for organization=bundesamt-fur-umwelt-bafu")
        # Process the 'packages' list as needed
        # For example, you could convert it to a pandas DataFrame:
        df = pd.DataFrame(packages)
    else:
        print("API request was not successful.")
        print(data['error'])
else:
    print(f"Error: Failed to retrieve data. Status code: {response.status_code}")

Found 342 entries for organization=bundesamt-fur-umwelt-bafu


In [54]:
dfOpendataSwiss = df

In [55]:
dfOpendataSwiss = dfOpendataSwiss[['keywords', 'title','description', 'modified']]
dfOpendataSwiss['description'] = dfOpendataSwiss['description'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
dfOpendataSwiss['title'] = dfOpendataSwiss['title'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
dfOpendataSwiss['Typ'] = 'Daten'
dfOpendataSwiss['Typ'] = dfOpendataSwiss.apply(lambda row: 'Geodatenmodell' if 'Geodatenmodell' in str(row['title']) else row['Typ'], axis=1)

/tmp/ipython-input-55-2487702280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOpendataSwiss['description'] = dfOpendataSwiss['description'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
/tmp/ipython-input-55-2487702280.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOpendataSwiss['title'] = dfOpendataSwiss['title'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)
/tmp/ipython-input-55-2487702280.py:4: SettingWithCopyWarning: 
A value is trying 

In [68]:
# prompt: wenn ein Wort aus dem Array Monitoring in title oder description vorkommt, wird das Attribut Typ auf Monitoring gesetzt

dfOpendataSwiss['Typ'] = dfOpendataSwiss.apply(
    lambda row: 'Monitoring' if any(word in str(row['title']) or word in str(row['description']) for word in Monitoring) else row['Typ'],
    axis=1
)

/tmp/ipython-input-68-2984563415.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfOpendataSwiss['Typ'] = dfOpendataSwiss.apply(


In [74]:

dfOpendataSwiss

,keywords,title,description,modified,Typ
0,"{'fr': ['inventaire-forestier-national', 'fore...",LFI: Themen nach NaiS-Waldformationen (10 Klas...,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T18:42:57+00:00,Monitoring
1,"{'fr': ['foret', 'sylviculture', 'inventaire-f...",LFI: Themen nach Baumzustand,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T17:09:00+00:00,Monitoring
2,"{'fr': ['foret', 'inventaire-forestier-nationa...",LFI: Themen nach Anzahl Schäden,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T17:17:16+00:00,Monitoring
3,"{'fr': ['foret', 'inventaire-forestier-nationa...",LFI: Themen nach Baumzustand (stehend/liegend),Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T17:48:45+00:00,Monitoring
4,"{'fr': ['inventaire-forestier-national', 'sylv...",LFI: Themen nach Schafthöhe gebrochener Dürrst...,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T18:01:59+00:00,Monitoring
...,...,...,...,...,...
337,"{'fr': ['sol', 'sol', 'atlas', 'interpolation'...",Geochemischer Bodenatlas der Schweiz: Quecksilber,Interpolierte Element-Konzentrationen (mg/kg F...,,Daten
338,"{'fr': ['environnement', 'observation-de-la-te...",NaiS-Weiserflächen und Beobachtungspunkte Suis...,Weiserflächen sind gemäss kantonalen Konzepten...,,Daten
339,"{'fr': ['sol', 'interpolation', 'metal-lourd',...",Geochemischer Bodenatlas der Schweiz: Thallium,Interpolierte Element-Konzentrationen (mg/kg F...,,Daten
340,"{'fr': [], 'de': [], 'en': [], 'it': []}",Grosse Hochwasserabflüsse in der Schweiz,Verzeichnis grosser Hochwasserabflüsse in schw...,2018-12-07T00:00:00,Daten


In [69]:
# prompt: gruppiere dfOpendataswiss nach Typ

print(dfOpendataSwiss.groupby('Typ').size())

Typ
Daten             282
Geodatenmodell      2
Monitoring         58
dtype: int64


In [72]:
dfStatistikenIndikatoren = pd.read_excel("StatistikenIndikatoren.xlsx")

In [77]:
# prompt: def map_options auf dfStatistikenIndikatoren anwenden

# Ensure 'Keywords (de)' column exists and is a list of strings, applying map_options
dfStatistikenIndikatoren['Keywords_kat'] = dfStatistikenIndikatoren['keywords'].apply(lambda x: [map_options(x)] if pd.notnull(x) and map_options(x) is not None else [])

#print(dfStatistikenIndikatoren[['Thema', 'Keywords (de)']].head())

In [78]:
dfStatistikenIndikatoren

,keywords,title,description,modified,Typ,Keywords_kat
0,Abfall,Abfallstatistik,Siedlungsabfälle und ihre Behandlung,2024-12-31,Statistik,[abfall]
1,Abfall,Sonderabfallstatistik,Sonderabfälle und ihre Behandlung,2024-12-31,Statistik,[abfall]
2,Klima,Treibhausgasinventar,Emissionen von teilhalogenierten Fluorkohlenwa...,2024-12-31,Statistik,[klima]
3,Luft,Landwirtschaftliche Betriebsparameter zur Bere...,Ausgestaltung und Nutzung von Stall_x0002_Syst...,NaT,Statistik,[luft]
4,Wald und Holz,Industrieholzerhebung,"Bezug, Verbrauch und Lager von Industrieholz",NaT,Statistik,[]
...,...,...,...,...,...,...
223,Wasser,Sanierung Fischgängigkeit (Fischabstieg),NaN,2024-10-08,Indikator,[wasser]
224,Wasser,Sanierung Fischgängigkeit (Fischaufstieg),NaN,2024-10-07,Indikator,[wasser]
225,Wasser,Sanierung Geschiebehaushalt Wasserkraft,NaN,2024-10-08,Indikator,[wasser]
226,Wasser,Sanierung Geschiebehaushalt Nicht-Wasserkrafta...,NaN,2024-10-08,Indikator,[wasser]


In [ ]:
# prompt: StatistikenIndikatoren.xlsx aus Dateien als Dataframe laden

# Assuming the file "StatistikenIndikatoren.xlsx" is uploaded to your Google Colab environment
try:
    df_stats_ind = pd.read_excel("StatistikenIndikatoren.xlsx")
    print("Successfully loaded StatistikenIndikatoren.xlsx")
    print(df_stats_ind.head())
except FileNotFoundError:
    print("Error: StatistikenIndikatoren.xlsx not found. Please upload the file.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")

In [70]:
dfOpendataSwiss

,keywords,title,description,modified,Typ
0,"{'fr': ['inventaire-forestier-national', 'fore...",LFI: Themen nach NaiS-Waldformationen (10 Klas...,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T18:42:57+00:00,Monitoring
1,"{'fr': ['foret', 'sylviculture', 'inventaire-f...",LFI: Themen nach Baumzustand,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T17:09:00+00:00,Monitoring
2,"{'fr': ['foret', 'inventaire-forestier-nationa...",LFI: Themen nach Anzahl Schäden,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T17:17:16+00:00,Monitoring
3,"{'fr': ['foret', 'inventaire-forestier-nationa...",LFI: Themen nach Baumzustand (stehend/liegend),Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T17:48:45+00:00,Monitoring
4,"{'fr': ['inventaire-forestier-national', 'sylv...",LFI: Themen nach Schafthöhe gebrochener Dürrst...,Das Landesforstinventar (LFI) ist eine periodi...,2024-10-14T18:01:59+00:00,Monitoring
...,...,...,...,...,...
337,"{'fr': ['sol', 'sol', 'atlas', 'interpolation'...",Geochemischer Bodenatlas der Schweiz: Quecksilber,Interpolierte Element-Konzentrationen (mg/kg F...,,Daten
338,"{'fr': ['environnement', 'observation-de-la-te...",NaiS-Weiserflächen und Beobachtungspunkte Suis...,Weiserflächen sind gemäss kantonalen Konzepten...,,Daten
339,"{'fr': ['sol', 'interpolation', 'metal-lourd',...",Geochemischer Bodenatlas der Schweiz: Thallium,Interpolierte Element-Konzentrationen (mg/kg F...,,Daten
340,"{'fr': [], 'de': [], 'en': [], 'it': []}",Grosse Hochwasserabflüsse in der Schweiz,Verzeichnis grosser Hochwasserabflüsse in schw...,2018-12-07T00:00:00,Daten


In [ ]:
# prompt: von dfOpendataSwiss im Attribut "description" nur "de" behalten

dfOpendataSwiss['description'] = dfOpendataSwiss['description'].apply(lambda x: x['de'] if isinstance(x, dict) and 'de' in x else None)

In [ ]:
# prompt: von dfOpendataSwiss folgende Attribute behalten
# keywords
# description
# resources
# num_resources
# modified

dfOpendataSwiss = dfOpendataSwiss[['keywords', 'description', 'resources', 'num_resources', 'modified']]

In [ ]:
Behalten
keywords
description
resources
num_resources
modified

In [13]:
dfOpendataSwiss

,license_title,maintainer,issued,title_for_slug,qualified_relations,private,maintainer_email,num_tags,contact_points,keywords,...,accrual_periodicity,owner_org,modified,url,extras,title,revision_id,identifier,coverage,scheduled
0,None,Schweizerisches Landesforstinventar. Eidgenöss...,2023-06-22T06:02:52+00:00,lfi-themen-nach-nais-waldformationen-10-klassen,[],False,lfiweb@wsl.ch,12,"[{'email': 'wald@bafu.admin.ch', 'name': 'Abte...","{'fr': ['inventaire-forestier-national', 'fore...",...,http://publications.europa.eu/resource/authori...,1b7caf78-d2a8-4e5c-ab20-954de72e9493,2024-10-14T18:42:57+00:00,https://lfi.ch/resultate/resultateliste.php?be...,[{'value': 'nfi_C-2637@bundesamt-fur-umwelt-ba...,{'fr': 'IFN: Thème par formations forestières ...,8dc118af-fb7a-4070-8f82-8e79e83cf229,nfi_C-2637@bundesamt-fur-umwelt-bafu,NaN,NaN
1,None,Schweizerisches Landesforstinventar. Eidgenöss...,2023-06-22T05:30:59+00:00,lfi-themen-nach-baumzustand,[],False,lfiweb@wsl.ch,12,"[{'email': 'wald@bafu.admin.ch', 'name': 'Abte...","{'fr': ['foret', 'sylviculture', 'inventaire-f...",...,http://publications.europa.eu/resource/authori...,1b7caf78-d2a8-4e5c-ab20-954de72e9493,2024-10-14T17:09:00+00:00,https://lfi.ch/resultate/resultateliste.php?be...,[{'value': 'nfi_C-98@bundesamt-fur-umwelt-bafu...,"{'fr': 'IFN: Thème par état de l'arbre', 'en':...",80ccdf83-4c12-4fe0-9d6f-2682eba8ba94,nfi_C-98@bundesamt-fur-umwelt-bafu,NaN,NaN
2,None,Schweizerisches Landesforstinventar. Eidgenöss...,2023-06-22T05:33:45+00:00,lfi-themen-nach-anzahl-schaden,[],False,lfiweb@wsl.ch,12,"[{'email': 'wald@bafu.admin.ch', 'name': 'Abte...","{'fr': ['foret', 'inventaire-forestier-nationa...",...,http://publications.europa.eu/resource/authori...,1b7caf78-d2a8-4e5c-ab20-954de72e9493,2024-10-14T17:17:16+00:00,https://lfi.ch/resultate/resultateliste.php?be...,[{'value': 'nfi_C-653@bundesamt-fur-umwelt-baf...,"{'fr': 'IFN: Thème par nombre de dégâts', 'de'...",7c41e862-fdd9-4bfb-be15-11a4bec90447,nfi_C-653@bundesamt-fur-umwelt-bafu,NaN,NaN
3,None,Schweizerisches Landesforstinventar. Eidgenöss...,2023-06-22T05:43:48+00:00,lfi-themen-nach-baumzustand-stehend-liegend,[],False,lfiweb@wsl.ch,12,"[{'email': 'wald@bafu.admin.ch', 'name': 'Abte...","{'fr': ['foret', 'inventaire-forestier-nationa...",...,http://publications.europa.eu/resource/authori...,1b7caf78-d2a8-4e5c-ab20-954de72e9493,2024-10-14T17:48:45+00:00,https://lfi.ch/resultate/resultateliste.php?be...,[{'value': 'nfi_C-1266@bundesamt-fur-umwelt-ba...,{'fr': 'IFN: Thème par état de l'arbre (sur pi...,d43ce62e-166e-4e8c-ad61-3baa34eeba0a,nfi_C-1266@bundesamt-fur-umwelt-bafu,NaN,NaN
4,None,Schweizerisches Landesforstinventar. Eidgenöss...,2023-06-22T05:47:28+00:00,lfi-themen-nach-schafthohe-gebrochener-durrsta...,[],False,lfiweb@wsl.ch,12,"[{'email': 'wald@bafu.admin.ch', 'name': 'Abte...","{'fr': ['inventaire-forestier-national', 'sylv...",...,http://publications.europa.eu/resource/authori...,1b7caf78-d2a8-4e5c-ab20-954de72e9493,2024-10-14T18:01:59+00:00,https://lfi.ch/resultate/resultateliste.php?be...,[{'value': 'nfi_C-1805@bundesamt-fur-umwelt-ba...,{'fr': 'IFN: Thème par hauteur d'arbres secs s...,e1e2f283-8d63-4148-b37b-840e6d625805,nfi_C-1805@bundesamt-fur-umwelt-bafu,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,None,boden@bafu.admin.ch,2023-06-30T00:00:00,geochemischer-bodenatlas-der-schweiz-quecksilber,[],False,boden@bafu.admin.ch,0,"[{'email': 'boden@bafu.admin.ch', 'name': 'bod...","{'fr': ['sol', 'sol', 'atlas', 'interpolation'...",...,http://publications.europa.eu/resource/authori...,1b7caf78-d2a8-4e5c-ab20-954de72e9493,,https://www.agroscope.admin.ch/agroscope/de/ho...,"[{'key': 'harvest_object_id', 'value': 'fe34ae...",{'fr': 'Atlas géochimique des sols de la Suiss...,8c7b0734-3b9e-4813-8f1c-6c62d8ac31e2,b667f34f-0bcb-4d67-bdef-0657d9c278cd@bundesamt...,,NaN
338,None,None,2024-11-24T00:00:00,nais-weiserflachen-und-beobachtungspunkte-suis...,[],False,None,0,"[{

In [4]:
# Funktion fragt alle Datensätze von opendata.swiss nach einem Keyword ab

def search_datasets(keyword):
    # Basis-URL der CKAN API
    base_url = "https://opendata.swiss/api/3/action/package_search"

    # Parameter für die Abfrage
    params = {
        "fq": f"organization:{'bundesamt-fur-umwelt-bafu'}",
        "q": f"keywords_de:{keyword}"
    }

    # Anfrage an die API senden
    response = requests.get(base_url, params=params)
    data = response.json()
    data = data["result"]["results"]

    # Daten aus Json auslesen
    description = [s['description']['de'] for s in data]
    title = [s['title']['de'] for s in data]
    url0 = [s['resources'][0]['url'] for s in data]
    title0 = [s['resources'][0]['title']['de'] for s in data]
    url1 = [s['resources'][1]['url'] for s in data]
    title1 = [s['resources'][1]['title']['de'] for s in data]

    # Daten als Dataframe speichern
    dict = {'title': title, 'description': description, 'Link': url0}
    dfOpendataSwiss = pd.DataFrame(dict)

    return dfOpendataSwiss

In [5]:
search_datasets('wald')

,title,description,Link
0,LFI: Themen nach NaiS-Waldformationen (10 Klas...,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
1,LFI: Themen nach Baumzustand,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
2,LFI: Themen nach Anzahl Schäden,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
3,LFI: Themen nach Baumzustand (stehend/liegend),Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
4,LFI: Themen nach Schafthöhe gebrochener Dürrst...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=%23pragm...
5,LFI: Themen nach Zeitpunkt des letzten Eingrif...,Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
6,LFI: Themen nach Baumart (56 Klassen),Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
7,LFI: Themen nach eingeführte Baumart (baumförm...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...
8,LFI: Themen nach Eigentum (2 Klassen),Das Landesforstinventar (LFI) ist eine periodi...,https://visualize.admin.ch/en/browse?previous=...
9,LFI: Themen nach Brusthöhendurchmesser (10 Kla...,Das Landesforstinventar (LFI) ist eine periodi...,https://lindas.admin.ch/sparql/#query=PREFIX%2...


In [ ]:
def filter_df_by_thema(thema):
    url = "https://www.bafu.admin.ch/bafu/de/home/zustand/indikatoren.html"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

    try:
      soup = BeautifulSoup(response.content, "html.parser")
      table_body = soup.find("table", attrs={"class": "table"})

      if not table_body:
        print("Table not found on the webpage")
        return None

      rows = table_body.find_all("tr")
      data = []
      for row in rows:
          cells = row.find_all("td")
          if len(cells) >= 2:
              thema_cell = cells[0].text.strip()
              indikatorname = cells[1].text.strip()
              link_element = cells[1].find('a')
              href = link_element.get('href') if link_element else None
              data.append({"Thema": thema_cell, "Indikatorname": indikatorname, "Link": "https://www.bafu.admin.ch" + href if href else None})

      df = pd.DataFrame(data)
    except Exception as e:
      print(f"An error occurred while processing the webpage: {e}")
      return None

    # Filter the DataFrame using .isin() for multiple Thema values if needed
    if isinstance(thema, list):
      filtered_df = df[df["Thema"].isin(thema)]
    else:
      filtered_df = df[df["Thema"] == thema]

    return filtered_df


In [ ]:
filter_df_by_thema('Abfall')

,Thema,Indikatorname,Link
0,Abfall,Abfallverbrennung,https://www.bafu.admin.ch/bafu/de/home/themen/...
1,Abfall,Energieeffizienz bei der Abfallverbrennung,https://www.bafu.admin.ch/bafu/de/home/themen/...
2,Abfall,Verwertete Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
119,Abfall,Auf Deponien abgelagerte Abfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
120,Abfall,Begleitscheine Sonderabfalltransporte,https://www.bafu.admin.ch/bafu/de/home/themen/...
121,Abfall,Energieproduktion aus Abfällen,https://www.bafu.admin.ch/bafu/de/home/themen/...
122,Abfall,Exportierte Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
123,Abfall,Grenzüberschreitender Verkehr mit Abfällen,https://www.bafu.admin.ch/bafu/de/home/themen/...
124,Abfall,Importierte Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
125,Abfall,In der Schweiz anfallende Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
